## Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.14.0 openai==1.107.0 chromadb==1.0.21 llama-index-vector-stores-chroma==0.5.3 jedi==0.19.2 \
                llama-index-llms-openai==0.5.6 llama-index-finetuning==0.4.1 llama-index-embeddings-adapter==0.4.1 \
                llama-index-embeddings-huggingface==0.6.1 llama-index-embeddings-openai==0.5.1 llama-index-llms-azure-openai==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Download the Dataset


In [3]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl: 0.00B [00:00, ?B/s]

In [4]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

## LlamaIndex Document

In [5]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

### Splitting Dataset


In [6]:
import random

random.shuffle(doc)
split_index = int(len(doc) * 0.9)

# TRAIN_DOCs and VALIDATION_DOCs
TRAIN_DOCs = doc[:split_index]
VALIDATION_DOCs = doc[split_index:]

# Chunking


In [7]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import Document

# Now use the parser
parser = SimpleNodeParser.from_defaults(chunk_size=768, chunk_overlap=64)
TRAIN_NODEs = parser.get_nodes_from_documents(TRAIN_DOCs)
VALIDATION_NODEs = parser.get_nodes_from_documents(VALIDATION_DOCs)

print(len(TRAIN_NODEs), len(VALIDATION_NODEs))

2868 246


In [8]:
# Use a subset of the dataset if testing.

# Test with a few sample, processing dataset fully can be costly depanding on the size.
# NOTE: Checkpoints are provided in the lesson, so no need to run the code on full dataset.

testing =False

if testing:
    TRAIN_NODEs = TRAIN_NODEs[0:200]
    VALIDATION_NODEs = VALIDATION_NODEs[0:40]

# Generate Question


We use a Large Language Model (LLM) to produce questions for each chunk of the dataset. Then we can use these data to train the model to develop embeddings that more accurately represent the types of questions users may ask.


In [9]:
# # Use this block of code if you want to generate the questions for the dataset.
# # Uncomment the following code, and keep in mind to comment all the contents in the next coding block.

# from llama_index.finetuning import generate_qa_embedding_pairs
# from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-5-mini", additional_kwargs={'reasoning_effort':'minimal'})

# # Generate questions for each chunk.

# TRAIN_DATASET = generate_qa_embedding_pairs(TRAIN_NODEs, llm=llm, output_path="./train_dataset.json")

# VALIDATION_DATASET = generate_qa_embedding_pairs(VALIDATION_NODEs, llm=llm, output_path="./val_dataset.json")

In [10]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="jaiganesan/Embedding-model-fine-tuning-dataset", repo_type="dataset",local_dir="/content/")


from llama_index.finetuning import EmbeddingQAFinetuneDataset

# Load the pre-generated questions json files.
TRAIN_DATASET = EmbeddingQAFinetuneDataset.from_json("./train_dataset.json")
VALIDATION_DATASET = EmbeddingQAFinetuneDataset.from_json("./val_dataset.json")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

val_dataset.json: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

train_dataset.json: 0.00B [00:00, ?B/s]

# Load an Embedding Model


In [11]:
from llama_index.core.embeddings import resolve_embed_model

# Load an existing embedding model with a adapter layer on top.
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
import torch

# Finetune the adapter
finetune_engine = EmbeddingAdapterFinetuneEngine(
    dataset=TRAIN_DATASET,
    embed_model=base_embed_model,
    model_output_path="model_output_test",
    epochs=2,
    verbose=True,
    bias=True,
)

In [13]:
# Initiate the Finetuning process
finetune_engine.finetune()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.5561195611953735
> [Epoch 0] Current loss: 1.5454347133636475
> [Epoch 0] Current loss: 1.3019955158233643
> [Epoch 0] Current loss: 1.210197925567627
> [Epoch 0] Current loss: 1.039259672164917
> [Epoch 0] Current loss: 1.0869501829147339
> [Epoch 0] Current loss: 1.6084486246109009
> [Epoch 0] Current loss: 1.3948438167572021
> [Epoch 0] Current loss: 1.624601125717163
> [Epoch 0] Current loss: 1.833229660987854
> [Epoch 0] Current loss: 2.352271318435669
> [Epoch 0] Current loss: 2.365095376968384
> [Epoch 0] Current loss: 2.2668750286102295
> [Epoch 0] Current loss: 1.8664630651474
> [Epoch 0] Current loss: 2.370300769805908
> [Epoch 0] Current loss: 2.346954345703125
> [Epoch 0] Current loss: 2.4227185249328613
> [Epoch 0] Current loss: 2.3735404014587402
> [Epoch 0] Current loss: 2.294159412384033
> [Epoch 0] Current loss: 2.3603808879852295
> [Epoch 0] Current loss: 2.3185572624206543
> [Epoch 0] Current loss: 2.3356428146362305
> [Epoch 0] Current lo

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.5034737586975098
> [Epoch 1] Current loss: 1.4798123836517334
> [Epoch 1] Current loss: 1.2930066585540771
> [Epoch 1] Current loss: 1.163027286529541
> [Epoch 1] Current loss: 1.029118537902832
> [Epoch 1] Current loss: 1.0037193298339844
> [Epoch 1] Current loss: 1.5205395221710205
> [Epoch 1] Current loss: 1.3807651996612549
> [Epoch 1] Current loss: 1.6184269189834595
> [Epoch 1] Current loss: 1.802191972732544
> [Epoch 1] Current loss: 2.352696418762207
> [Epoch 1] Current loss: 2.3439197540283203
> [Epoch 1] Current loss: 2.2643284797668457
> [Epoch 1] Current loss: 1.8568460941314697
> [Epoch 1] Current loss: 2.3480916023254395
> [Epoch 1] Current loss: 2.3382058143615723
> [Epoch 1] Current loss: 2.3973588943481445
> [Epoch 1] Current loss: 2.3641152381896973
> [Epoch 1] Current loss: 2.27299165725708
> [Epoch 1] Current loss: 2.3372747898101807
> [Epoch 1] Current loss: 2.3109443187713623
> [Epoch 1] Current loss: 2.3237788677215576
> [Epoch 1] Curr

In [14]:
embed_model = finetune_engine.get_finetuned_model()

# Or, import model from the directory whenever required.
# from llama_index.core.embeddings import LinearAdapterEmbeddingModel
# embed_model = LinearAdapterEmbeddingModel(base_embed_model, "model_output_test")

In [15]:
embed_model

AdapterEmbeddingModel(model_name='Adapter for BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7a3330892030>, num_workers=None, embeddings_cache=None)

## Fine tuning OpenAI Embedding Model using Adapter method

In [16]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.embeddings.openai import OpenAIEmbedding

openai_finetune_engine = EmbeddingAdapterFinetuneEngine(
    dataset=TRAIN_DATASET,
    embed_model=OpenAIEmbedding(model="text-embedding-3-small"),
    model_output_path="model_output_test_openai",
    bias=True,
    epochs=2,
    verbose=True,
)

In [17]:
openai_finetune_engine.finetune()

openai_embed_model = openai_finetune_engine.get_finetuned_model()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.1989820003509521
> [Epoch 0] Current loss: 1.2278980016708374
> [Epoch 0] Current loss: 0.9621146321296692
> [Epoch 0] Current loss: 0.9324359893798828
> [Epoch 0] Current loss: 1.1187877655029297
> [Epoch 0] Current loss: 0.9118953943252563
> [Epoch 0] Current loss: 0.8484293818473816
> [Epoch 0] Current loss: 1.1641426086425781
> [Epoch 0] Current loss: 1.1628453731536865
> [Epoch 0] Current loss: 1.7471033334732056
> [Epoch 0] Current loss: 2.292062520980835
> [Epoch 0] Current loss: 2.366800308227539
> [Epoch 0] Current loss: 2.324587106704712
> [Epoch 0] Current loss: 1.804469108581543
> [Epoch 0] Current loss: 2.360020160675049
> [Epoch 0] Current loss: 2.3147666454315186
> [Epoch 0] Current loss: 2.337958574295044
> [Epoch 0] Current loss: 2.272594451904297
> [Epoch 0] Current loss: 1.4585193395614624
> [Epoch 0] Current loss: 2.3004822731018066
> [Epoch 0] Current loss: 2.2815310955047607
> [Epoch 0] Current loss: 2.3898911476135254
> [Epoch 0] Curre

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.1398003101348877
> [Epoch 1] Current loss: 1.1968791484832764
> [Epoch 1] Current loss: 0.9486591219902039
> [Epoch 1] Current loss: 0.9087713360786438
> [Epoch 1] Current loss: 1.121414303779602
> [Epoch 1] Current loss: 0.9038141965866089
> [Epoch 1] Current loss: 0.829842746257782
> [Epoch 1] Current loss: 1.1796451807022095
> [Epoch 1] Current loss: 1.1539713144302368
> [Epoch 1] Current loss: 1.6840966939926147
> [Epoch 1] Current loss: 2.284733295440674
> [Epoch 1] Current loss: 2.3604114055633545
> [Epoch 1] Current loss: 2.316593647003174
> [Epoch 1] Current loss: 1.7934720516204834
> [Epoch 1] Current loss: 2.3495090007781982
> [Epoch 1] Current loss: 2.30342698097229
> [Epoch 1] Current loss: 2.334984302520752
> [Epoch 1] Current loss: 2.253394842147827
> [Epoch 1] Current loss: 1.4440841674804688
> [Epoch 1] Current loss: 2.288766860961914
> [Epoch 1] Current loss: 2.2726950645446777
> [Epoch 1] Current loss: 2.3813390731811523
> [Epoch 1] Current

In [18]:
openai_embed_model

AdapterEmbeddingModel(model_name='Adapter for text-embedding-3-small', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7a3329487770>, num_workers=None, embeddings_cache=None)

# Evaluate


## Define the Evaluation Functions


In [19]:
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.core import Settings
from tqdm import tqdm
import pandas as pd

def evaluate(dataset, embedding_model, top_k=5, verbose=False):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    Settings.embed_model = embedding_model

    # Chunking the documents and generating embeddings
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(nodes, show_progress=True)

    # Define a retriever to answer the questions
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []

    # Look into each response sources to see if the chunk that contains the answer is retrieved.
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]

        try:
            rank = retrieved_ids.index(expected_id) + 1
            reciprocal_rank = 1 / rank
        except ValueError:
            rank = None
            reciprocal_rank = 0

        is_hit = expected_id in retrieved_ids

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
            "rank": rank,
            "reciprocal_rank": reciprocal_rank
        }
        eval_results.append(eval_result)

    return eval_results

## OpenAI Embedding Model Evaluation


In [20]:
# Load the OpenAI Ada model and evaluate it.
openai_text_embedding_small = OpenAIEmbedding(model="text-embedding-3-small")
openai_embedding_val_results = evaluate(VALIDATION_DATASET, embedding_model=openai_text_embedding_small)

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [02:27<00:00,  3.35it/s]


In [21]:
openai_embedding_val_results = [
    result for result in openai_embedding_val_results if isinstance(result, dict)
]

df_openai = pd.DataFrame(openai_embedding_val_results)

hit_rate_openai = df_openai["is_hit"].mean()
mrr_openai = df_openai["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai}")
print(f"MRR: {mrr_openai}")

Hit rate: 0.9354838709677419
MRR: 0.787869623655914


### OpenAI Embedding Model with Fine Tuned Adapter Model Evaluation

In [22]:
from llama_index.embeddings.adapter import AdapterEmbeddingModel

openai_embed_model = AdapterEmbeddingModel(openai_text_embedding_small, "model_output_test_openai")

val_results_ft_openai = evaluate(VALIDATION_DATASET, embedding_model = openai_embed_model)

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [02:38<00:00,  3.13it/s]


In [23]:
val_results_ft_openai = [
    result for result in val_results_ft_openai if isinstance(result, dict)
]

df_openai_ft = pd.DataFrame(val_results_ft_openai)

hit_rate_openai_ft = df_openai_ft["is_hit"].mean()
mrr_openai_ft = df_openai_ft["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai_ft}")
print(f"MRR: {mrr_openai_ft}")

Hit rate: 0.9637096774193549
MRR: 0.8210685483870968


## Open Source BAAI Model Evaluation


In [24]:
# Load the Base model without fine-tuning
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
bge_val_results = evaluate(VALIDATION_DATASET, embedding_model=base_embed_model)

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [00:39<00:00, 12.68it/s]


In [25]:
bge_val_results = [
    result for result in bge_val_results if isinstance(result, dict)
]

df_bge = pd.DataFrame(bge_val_results)

hit_rate_bge = df_bge["is_hit"].mean()
mrr_bge = df_bge["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_bge}")
print(f"MRR: {mrr_bge}")

Hit rate: 0.8387096774193549
MRR: 0.7207325268817205


## FineTuned BAAI Adapter Embedding Model Evaluation


In [28]:
from llama_index.embeddings.adapter import AdapterEmbeddingModel

# Load the Fine-tuned model.
embed_model = AdapterEmbeddingModel(base_embed_model, "model_output_test")

val_results_finetuned = evaluate(VALIDATION_DATASET, embedding_model=embed_model)

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [00:36<00:00, 13.45it/s]


In [29]:
val_results_finetuned = [
    result for result in val_results_finetuned if isinstance(result, dict)
]

df_finetuned = pd.DataFrame(val_results_finetuned)

hit_rate_finetuned = df_finetuned["is_hit"].mean()
mrr_finetuned = df_finetuned["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_finetuned}")
print(f"MRR: {mrr_finetuned}")

Hit rate: 0.8629032258064516
MRR: 0.7499663978494624
